In [ ]:
import librosa
import numpy as np
import parselmouth
from scipy.stats import iqr, skew, kurtosis
from scipy.signal import find_peaks
from pathlib import Path
import pandas as pd
import noisereduce as nr
import math
import re


ravdess_folder = Path(r"C:/Users/jbkee/OneDrive/Desktop/Jupyter Projects/RAVDESS/audio_speech_actors_01-24")
savee_folder = Path(r"C:/Users/jbkee/OneDrive/Desktop/Jupyter Projects/SAVEE/AudioData")
emodb_folder = Path(r"C:/Users/jbkee/OneDrive/Desktop/Jupyter Projects/EMODB")

def safe_formant_values(formant_obj, formant_index, duration):
    times = np.arange(0, duration, 0.01)
    values = [formant_obj.get_value_at_time(formant_index, t) for t in times]
    return [v for v in values if v is not None and not np.isnan(v) and v > 0]

def extract_emotion_from_ravdess_filename(filename):
    emotion_map = {
        "01": "neutral",
        "03": "happy",
        "04": "sad",
        "05": "angry",
        "06": "fearful",
        "07": "disgust",
    }
    
    emotion = filename.split("-")[2]
    
    return emotion_map.get(emotion, "unknown")

def extract_emotion_from_savee_filename(filename):
    emotion_map = {
        "a": "angry",
        "d": "disgust",
        "f": "fearful",
        "h": "happy",
        "n": "neutral",
        "sa": "sad",
    }
    
    emotion = filename[0].lower()
    
    if filename[:2].lower() in emotion_map:
        return emotion_map[filename[:2].lower()]
    return emotion_map.get(emotion, "unknown")

def extract_emotion_from_emodb_filename(filename):
    emotion_map = {
        "w": "angry",
        "e": "disgust",
        "a": "fearful",
        "f": "happy",
        "n": "neutral",
        "t": "sad",
    }
    
    emotion = filename[5].lower()

    return emotion_map.get(emotion, "unknown")

def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)

    # Trim non-speech segments from beginning and end
    y, _ = librosa.effects.trim(y, top_db=20)

    # Normalize audio
    if np.max(np.abs(y)) > 0:
        y = y / np.max(np.abs(y))

    # Noise reduction using a noise profile from the first 0.25 seconds
    noise_sample = y[:int(0.25 * sr)]
    y = nr.reduce_noise(y=y, sr=sr, y_noise=noise_sample, prop_decrease=1.0)

    # Extract MFCCs (first 13 coefficients)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    delta_mfccs = librosa.feature.delta(mfccs)
    delta2_mfccs = librosa.feature.delta(mfccs, order=2)

    # Extract additional spectral features
    zcr = librosa.feature.zero_crossing_rate(y)[0]
    centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
    flatness = librosa.feature.spectral_flatness(y=y)[0]
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)

    # Extract formants using parselmouth
    snd = parselmouth.Sound(y, sampling_frequency=sr)
    formant = snd.to_formant_burg()
    duration = snd.duration
    formants = [safe_formant_values(formant, i + 1, duration) for i in range(3)]

    # Extract pitch
    pitch = snd.to_pitch()
    pitches = pitch.selected_array['frequency'][pitch.selected_array['frequency'] > 0]

    # Extract harmonicity (HNR)
    harmonicity = snd.to_harmonicity_cc()
    hnr_values = harmonicity.values
    hnr_values = hnr_values[~np.isnan(hnr_values)]

    # Extract energy (RMS)
    energy = librosa.feature.rms(y=y)[0]

    # Statistical descriptors
    def compute_stats(data):
        skew_val = skew(data) if len(data) > 2 else 0
        kurt_val = kurtosis(data) if len(data) > 2 else 0
        skew_val = 0 if math.isnan(skew_val) else skew_val
        kurt_val = 0 if math.isnan(kurt_val) else kurt_val
        peak_count = len(find_peaks(data)[0]) if len(data) > 2 else 0
        range_val = (np.max(data) - np.min(data)) if len(data) > 0 else 0
        return {
            'mean': np.mean(data) if len(data) > 0 else 0,
            'median': np.median(data) if len(data) > 0 else 0,
            'std_dev': np.std(data) if len(data) > 0 else 0,
            'min': np.min(data) if len(data) > 0 else 0,
            'max': np.max(data) if len(data) > 0 else 0,
            'q1': np.percentile(data, 25) if len(data) > 0 else 0,
            'q3': np.percentile(data, 75) if len(data) > 0 else 0,
            'iqr': iqr(data) if len(data) > 0 else 0,
            'jitter': np.mean(np.abs(np.diff(data))) if len(data) > 1 else 0,
            'shimmer': np.std(np.abs(np.diff(data))) if len(data) > 1 else 0,
            'skewness': skew_val,
            'kurtosis': kurt_val,
            'peak_count': peak_count,
            'range': range_val
        }

    features = {}

    # Compute stats for MFCCs and their deltas
    for i in range(13):
        mfcc_stats = compute_stats(mfccs[i])
        delta_stats = compute_stats(delta_mfccs[i])
        delta2_stats = compute_stats(delta2_mfccs[i])
        for stat, value in mfcc_stats.items():
            features[f'mfcc_{i+1}_{stat}'] = value
        for stat, value in delta_stats.items():
            features[f'mfcc_{i+1}_delta_{stat}'] = value
        for stat, value in delta2_stats.items():
            features[f'mfcc_{i+1}_delta2_{stat}'] = value

    # Compute stats for spectral features
    for name, array in zip(
        ['zcr', 'spectral_centroid', 'spectral_bandwidth', 'spectral_rolloff', 'spectral_flatness'],
        [zcr, centroid, bandwidth, rolloff, flatness]
    ):
        stats = compute_stats(array)
        for stat, value in stats.items():
            features[f'{name}_{stat}'] = value

    for i in range(contrast.shape[0]):
        stats = compute_stats(contrast[i])
        for stat, value in stats.items():
            features[f'spectral_contrast_{i+1}_{stat}'] = value

    # Compute stats for Formants
    for i in range(3):
        formant_stats = compute_stats(formants[i])
        for stat, value in formant_stats.items():
            features[f'formant_{i+1}_{stat}'] = value

    # Compute stats for Pitch
    pitch_stats = compute_stats(pitches)
    for stat, value in pitch_stats.items():
        features[f'pitch_{stat}'] = value

    # Compute stats for Energy
    energy_stats = compute_stats(energy)
    for stat, value in energy_stats.items():
        features[f'energy_{stat}'] = value

    # Compute stats for Harmonicity (HNR)
    hnr_stats = compute_stats(hnr_values)
    for stat, value in hnr_stats.items():
        features[f'harmonicity_{stat}'] = value

    return features

# Collect and save features
all_features = []

# EMODB
for file in emodb_folder.glob("*.wav"):
    emotion = extract_emotion_from_emodb_filename(file.name)
    if (emotion != "unknown"):
        features = extract_features(file)
        features["emotion"] = emotion
        all_features.append(features)
        print(f"Processed {file.name} with {len(features)} features")
    else:
        print(f"Failed to process {file.name}. Emotion unknown")

# SAVEE
for file in savee_folder.glob("**/*.wav"):
    emotion = extract_emotion_from_savee_filename(file.name)
    if (emotion != "unknown"):
        features = extract_features(file)
        features["emotion"] = emotion
        all_features.append(features)
        print(f"Processed {file.name} with {len(features)} features")
    else:
        print(f"Failed to process {file.name}. Emotion unknown")

# RAVDESS
for file in ravdess_folder.glob("**/*.wav"):
    emotion = extract_emotion_from_ravdess_filename(file.name)
    if (emotion != "unknown"):
        features = extract_features(file)
        features["emotion"] = emotion
        all_features.append(features)
        print(f"Processed {file.name} with {len(features)} features")
    else:
        print(f"Failed to process {file.name}. Emotion unknown")


# Save to CSV
df = pd.DataFrame(all_features)
df.to_csv("combined_features_2.csv", index=False)
print("All features saved to combined_features_2.csv")